In [1]:
from vtk import vtkUnstructuredGridReader
from vtk import vtkXMLUnstructuredGridReader
from vtk.util import numpy_support as VN
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# A python code which computes the maximum and minimum sigma_xx at along a point near the attachment 
# point of the beamm. The analytic value of the maximum and minimum values from Geodynamics 3rd edition
# by Turcotte and Schubert (equation 3.86) are also shown on the figure for comparison. This plot is meant
# to be used as a means for choosing the solution file to run in the other python scripts which generates
# the stress profiles and the beam image. This code iterates over all .vtu files output by ASPECT by default,
# so it may take a while for it to run.
# The arguments of the function are:
#    sol_dir_path - the pathway to the directory which contains all of the solution.vtu files
#    rho_beam - the density of the beam set in the input file
#    rho_amb - the density of the ambient material surrounding the beam set in the input file
#    mu - the elastic shear modulus of the beam that is set in the input file
#    nu - the poissons ratio, 0.5 for incompressible
#    g - the gravity set in the input file, set it to be negative
#    beamtop - the top of the beam composition that is set in the input file
#    beambot - the bottom of the beam composition that is set in the input file
#    L - the length of the beam that is sent in the input file
#    x_spacing - the spacing of the base mesh without refinements, this is needed to make sure that 
#                the vertical profile is taken at a value where there are actually mesh points
######################################################################################################

def stress_time(sol_dir_path, rho_beam, rho_amb, mu, nu, g, beamtop, beambot, L, x_spacing):
    solutions = os.listdir( sol_dir_path ) # list all the files in the solutions directory
    maxvals = []
    minvals = []
    
    # iterate over EVERY solution.vtu file output by ASPECT
    for j, soln in enumerate(solutions):
        if j%3==0 and j>0: # only choose every 3rd, ASPECT outputs a .vtu, .pvtu and a .visit for each output, 
                           # only the .vtu file is needed
            filepath = os.path.join(sol_dir_path, soln) # join solution directory path with .vtu file 
            
            # Read in the data from the .vtu file
            reader = vtkXMLUnstructuredGridReader()
            reader.SetFileName(filepath)
            reader.Update()
            data = reader.GetOutput()
            points = data.GetPoints()
            x = vtk_to_numpy(points.GetData())
            sigxx = vtk_to_numpy(data.GetPointData().GetArray('ve_stress_xx'))
            pressure = vtk_to_numpy(data.GetPointData().GetArray('p'))
            
            # create arrays that take values along a vertical profile near attachment point of the beam
            sigxx_prof = []
            pressure_prof = []
            y_prof = []
            for n in range(len(sigxx)):
                if x[:, 0][n] == 2*x_spacing and x[:, 1][n] not in y_prof:
                    sigxx_prof.append(sigxx[n])
                    pressure_prof.append(pressure[n])
                    y_prof.append(x[:, 1][n])
            
            # to get sigma, we need to subtract out the lithostatic pressure from ASPECTs sigmaxx output
            ymax = max(y_prof)
            pressure_lith = []
            for y_val in y_prof:
                pressure_lith.append(rho_amb * -g * (ymax - y_val))
        
            # create arrays with max and min stresses along the vertical profile
            sigprime = np.array(sigxx_prof) - np.array(pressure_prof) + np.array(pressure_lith)
            maxvals.append(max(sigprime))
            minvals.append(min(sigprime))
    
    # set up parameters to get the analytic value of max stress
    drho = abs(rho_amb - rho_beam)
    h = abs(beamtop - beambot)
    E = 2 * mu * (1 + nu)
    D = E * h**3 / (12 * (1 - nu**2))
    q = drho * g * h
    sigmax = 3 * q * L**2 / h**2
    output_num = np.arange(0, len(maxvals))
    
    # Plot
    plt.figure(dpi = 140)
    plt.scatter(output_num, np.array(maxvals) / 1e6, label = 'Max', s = 10)
    plt.scatter(output_num, np.array(minvals) / 1e6, label = 'Min', s = 10)
    plt.axhline(-sigmax / 1e6, color = 'k', linestyle = '--', label = 'Analytic Max/Min')
    plt.axhline(sigmax / 1e6, color = 'k', linestyle = '--')
    plt.xlabel('Output Number')
    plt.ylabel('Max $\sigma_{xx}$ - MPa')
    plt.title('Max $\sigma_{xx}$ with Time')
    plt.legend()
    plt.show()